In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [58]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix

In [3]:
np.random.seed(20200908)

In [148]:
water = pd.read_csv('./data/Potomac_transformed.csv')

**Keep only consecutive days in the dataframe**

In [149]:
# transform Date_Time into datetime object
water['Date_Time'] = pd.to_datetime(water['Date_Time'])

# source: https://www.dataquest.io/blog/python-datetime-tutorial/

In [150]:
# sort data by Date_Time column
water.sort_values(by=['Date_Time'], inplace=True)

In [151]:
def difference(column):
    differences = []
    for i in range(1, column.shape[0]):
        timediff = column[i] - column[i - 1]
        differences.append(timediff)
    return pd.Series(differences)

# source: https://machinelearningmastery.com/difference-time-series-dataset-python/

In [152]:
# create new column with differenced Date_Time
water['TimeDiff'] = difference(water['Date_Time'])

In [153]:
water['TimeDiff'].describe()

count                     29773
mean     0 days 04:15:16.807174
std      1 days 02:02:17.224628
min             0 days 00:00:00
25%             0 days 00:00:00
50%             0 days 00:00:00
75%             0 days 00:00:00
max            27 days 03:27:00
Name: TimeDiff, dtype: object

In [154]:
water['TimeDiff'].isna().sum()

1

In [155]:
# keep only consecutive days
consec = water.loc[water['TimeDiff'].dt.days < 2]

In [156]:
consec.shape

(29063, 48)

In [157]:
consec['TimeDiff'].describe()

count                     29063
mean     0 days 00:47:35.100299
std      0 days 04:10:44.732928
min             0 days 00:00:00
25%             0 days 00:00:00
50%             0 days 00:00:00
75%             0 days 00:00:00
max             1 days 23:50:00
Name: TimeDiff, dtype: object

In [158]:
consec.columns

Index(['Date_Time', 'SampleId', 'SampleDepth', 'Agency', 'Cruise', 'Database',
       'HUC12', 'Latitude', 'Layer', 'Longitude', 'Method', 'Program',
       'Project', 'SampleReplicateType', 'SampleType', 'Source', 'Station',
       'TierLevel', 'Unit', 'Point', 'HUC12_', 'HUCNAME_', 'FIPS_', 'COUNTY_',
       'STATE_', 'TotalDepth', 'Parameter_CHLA', 'Parameter_DO',
       'Parameter_NH4F', 'Parameter_NO3F', 'Parameter_PH', 'Parameter_PO4F',
       'Parameter_SALINITY', 'Parameter_SECCHI', 'Parameter_TALK',
       'Parameter_TDS', 'Parameter_TKNW', 'Parameter_TN', 'Parameter_TP',
       'Parameter_TSS', 'Parameter_TURB_NTU', 'Parameter_WTEMP',
       'tide_Flood Tide', 'tide_High Slack Tide', 'tide_Low Slack Tide',
       'Years', 'Months', 'TimeDiff'],
      dtype='object')

In [159]:
# columns to drop
col_to_drop = ['Agency', 'Cruise', 'Database', 'HUC12', 'Latitude', 'Layer', 'Longitude', 'Method', 'Program', 'Project', 
               'SampleReplicateType', 'SampleType', 'Source', 'Station', 'TierLevel', 'Unit', 'Point', 'FIPS_', 'COUNTY_', 'STATE_',
               'HUCNAME_']

In [160]:
consec = consec.drop(columns=col_to_drop)

In [206]:
# check sample depth vs chlorophyll
consec.groupby(by='SampleDepth')['Parameter_CHLA'].mean()

SampleDepth
0.000      1.498017
0.080      0.000000
0.090      0.000000
0.098      0.800000
0.100      3.358993
            ...    
19.400     3.920000
19.500     5.383333
19.600     8.540000
19.700     2.140000
20.000    13.146810
Name: Parameter_CHLA, Length: 363, dtype: float64

In [207]:
# check sample depth vs. total phosphorus
consec.groupby(by='SampleDepth')['Parameter_TP'].mean()

SampleDepth
0.000     0.030851
0.080     0.000000
0.090     0.000000
0.098     0.000000
0.100     0.000419
            ...   
19.400    0.140700
19.500    0.146100
19.600    0.060800
19.700    0.034100
20.000    0.032557
Name: Parameter_TP, Length: 363, dtype: float64

In [315]:
# group data by sample id and get mean

data = consec.groupby(by='SampleId').agg('mean').copy()

In [316]:
data.isnull().sum().sum()

0

In [317]:
data.shape

(7082, 24)

In [318]:
# reset index 
data.reset_index(inplace=True)

In [319]:
data.columns

Index(['SampleId', 'SampleDepth', 'HUC12_', 'TotalDepth', 'Parameter_CHLA',
       'Parameter_DO', 'Parameter_NH4F', 'Parameter_NO3F', 'Parameter_PH',
       'Parameter_PO4F', 'Parameter_SALINITY', 'Parameter_SECCHI',
       'Parameter_TALK', 'Parameter_TDS', 'Parameter_TKNW', 'Parameter_TN',
       'Parameter_TP', 'Parameter_TSS', 'Parameter_TURB_NTU',
       'Parameter_WTEMP', 'tide_Flood Tide', 'tide_High Slack Tide',
       'tide_Low Slack Tide', 'Years', 'Months'],
      dtype='object')

In [320]:
# shift Parameter_CHLA by 1 day compared to everything else
data['Parameter_CHLA'] = data['Parameter_CHLA'].shift(periods=-1)

In [321]:
data['Parameter_CHLA'].tail(10)

7072    26.800000
7073     0.000000
7074     1.500000
7075     0.000000
7076     0.100000
7077     0.000000
7078     3.360000
7079     0.997000
7080     4.663667
7081          NaN
Name: Parameter_CHLA, dtype: float64

In [322]:
# drop first row - where Parameter_CHLA became NaN
data.dropna(inplace=True)

**Model**<br>
**Predicting Chlorophyll level one day after other parameter values were measured**

In [323]:
# train_test_split
# training data = up until last two years
# testing data = last two years (2018-2019)

train = data.loc[data['Years'] < 2018]
test = data.loc[data['Years'] >= 2018]

In [324]:
train.shape

(6468, 25)

In [325]:
test.shape

(613, 25)

In [326]:

X_train = train.drop(columns=['SampleId', 'SampleDepth', 'TotalDepth', 'Parameter_CHLA'])
y_train = train['Parameter_CHLA']

X_test = test.drop(columns=['SampleId', 'SampleDepth', 'TotalDepth', 'Parameter_CHLA'])
y_test = test['Parameter_CHLA']

In [327]:
# instantiate random forest classifier
rf = RandomForestRegressor()

In [328]:
# parameters for grid search
params = {
    'n_estimators'      : [100, 200, 300],
    'min_samples_split' : [20],
    'max_depth'         : [5]
}

In [329]:
# Set up grid search
gs = GridSearchCV(rf, 
                  param_grid=params, 
                  return_train_score=True, 
                  cv = 5)

In [330]:
# Run grid search on training data with a time check. 
import time
t0 = time.time()
gs.fit(X_train, y_train)
print(time.time() - t0)

58.695672035217285


**Evaluation**

In [331]:
# Score on training set.
r2_train = gs.score(X_train, y_train)
r2_train

0.3197732237005665

In [332]:
# Score on testing set.
r2_test = gs.score(X_test, y_test)
r2_test

-0.017203156284911092

In [333]:
# List best hyperparameters found in Grid Search
gs.best_params_

{'max_depth': 5, 'min_samples_split': 20, 'n_estimators': 300}

In [334]:
# 5-fold cross validation score.
gs.best_score_

-0.15520158260432632

Predicting chlorophyll levels 1 day later than measuring all other feature parameters:<br>
Train_R2: 0.3 <br>
Test_R2: -0.015 <br>
Cross_validated_R2: -0.14 <br>
<br>
Predicting chlorophyll levels 3 day later than measuring all other feature parameters:<br>
Train_R2: 0.2 <br>
Test_R2: -0.05 <br>
Cross_validated_R2: -0.17 <br>
<br>
Predicting chlorophyll levels 7 day later than measuring all other feature parameters:<br>
Train_R2: 0.27<br>
Test_R2: 0.044<br>
Cross_validated_R2: -0.14<br>